# <center>人物チェックによるエイムチェックシステム</center>

参考サイト:
https://teratail.com/questions/282111

In [1]:
import torch
import torchvision
from PIL import Image, ImageDraw
from torchvision import models as models
from torchvision import transforms as transforms
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# モデルを作成する。
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True).to(device)
model.eval()

/Users/msz/miniforge3/envs/env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [2]:
# 中心を切り取る関数
def pick_center(default_img, pick_wide, pick_high):
    default_wide, default_high = default_img.size
    x_min = (default_wide - pick_wide) // 2
    y_min = (default_wide - pick_high) // 2
    x_max = x_min + pick_wide
    y_max = y_min + pick_high
    return default_img.crop((x_min,y_min,x_max,y_max))

# 
def detection(file_name,score):
    # 画像の読み込み
    image = Image.open(file_name).convert('RGB')
    image = pick_center(image,800,600)
    image_tensor = transforms.functional.to_tensor(image).to(device)

    # 推論の実行
    out = model([image_tensor])[0]

    # 推論された結果から人である確率が0.4(score=0.4)であるという条件でボックスのフィルタリング
    ID = 1
    boxes = out["boxes"][out["labels"] == ID & out["scores"] >= score].detach().cpu().numpy()
    
    # 画像に図形などを描画するための準備
    draw = ImageDraw.Draw(image, mode="RGBA")

    high = 0
    wide   = 0
    # 検出された人物のボックスを描画
    for box in boxes:
        # 
        high = (box[0] + box[1]) / 2 
        wide = (box[2] + box[3]) / 2
        # ボックスの描画
        draw.rectangle(box, outline="red", width=3)

    plt.imshow(image)
    plt.show()
    
    return

### 画像の出力

In [ ]:
detection("sample.PNG",0.8)

In [ ]:
detection("sample1.PNG",0.8)

In [ ]:
detection("sample.PNG",0.4)

In [ ]:
detection("sample1.PNG",0.4)